# Recipe: ROTI

In this recipe we will replicate Figure 2b of {cite:p}`juan2017method` (for the SEY1 station and GPS PRN26) using the methodology described in {prf:ref}`roti`.

This recipe includes the following steps:

- Loading a Rinex file into a `pandas` DataFrame for later processing
- Computing the LI for a satellite
- Computing the $\Delta STEC$ for a satellite
- Computing the ROTI

First we will import some necessary modules:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from roktools import rinex

# Import various classes that will be used to filter DataFrame
from roktools.gnss.types import ConstellationId, TrackingChannel, Satellite

# Add the path so that we can import the custom code of this Jupyter book
import sys
sys.path.append('../source/')

# Import methods from the custom code of this book
from gnss.observables import compute_geometry_free
from gnss.edit import mark_time_gap
from helpers import compute_decimal_hours

%matplotlib widget

## Load a Rinex

The `roktools` module contains a utility method that loads a RINEX file into a convenient data structure (`pandas` DataFrame) that eases data analysis. 

In [ ]:
rinex_file = '../assets/SEYG00SYC_R_20140581500_05H_01S_MO.rnx'

df = rinex.to_dataframe(rinex_file)

The contents of the `DataFrame` are layout as a data table, with various columns (pseudorange, phase, Doppler, ...).
To peek the first contents of the RINEX file and check how the data is organized and which are the differents columns of the table, use the `head` function. 

In [ ]:
df.head()

Within each column, data is stored as Python classes. For instance, the column `sat` contains the satellite, but it is not stored as a `string`, but as a `Satellite` object. To check the class type, use the following command


In [ ]:
df['sat'].dtype

Unfortunately, the `O` identifier tells us that the `sat` is an object, but we do not have information in the specific class. To know the specific class type. Do this instead:

In [ ]:
df['sat'].apply(type).head()

Now that the whole RINEX is loaded into the `DataFrame`, you can perform some basics checks such as getting the e.g. list of the satellites contained in the file:

In [ ]:
df['sat'].unique()

You can also select the data for just one satellite

In [ ]:

# Create a Satellite object, that will be used for the DataFrame indexing
sat = Satellite.from_string('G26')

# Create a "sub-DataFrame" with the contents for this satellite only
df_sat = df[df['sat'] == sat]
df_sat.head()

## Compute the ionospheric combination

In this section we will compute the ionospheric (or geometry-free) combination (see equation [](iono_comb_phase)) for a specific satellite and data combination. To do this, we will use [`pandas` merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html). The `merge` function allow us to join two `DataFrame`s based on the contents of a column in an efficient, vectorized manner, without cumbersome `for` loops, which typically slow down Python code (and in general should be avoided)

First let's check which are the different tracking channels available for a the previous satellite:

In [ ]:
df_sat['channel'].unique()

To compute the ionospheric combination we need to pick two channels, for this example we will pick the observables generated with the C/A tracking loop at the L1 frequency (RINEX code `1C`) and the encrypted code at the L2 (RINEX code `2W`, usually obtained with proprietary techniques such as *semi-codeless* tracking)

In [ ]:

ch_a = TrackingChannel.from_string('1C')
ch_b = TrackingChannel.from_string('2W')

# Select the channels that will be used for this recipe
df_sat_ch_a = df_sat[df_sat['channel'] == str(ch_a)]
df_sat_ch_b = df_sat[df_sat['channel'] == str(ch_b)]

# Check the results
df_sat_ch_a.head()

Now use the `merge` method to compute the LI combination

In [ ]:
df_sat_li = pd.merge(df_sat_ch_a, df_sat_ch_b, on='epoch', how='inner', suffixes=('_a', '_b'))
df_sat_li.tail()

The RINEX format specifies the carrier phase as cycles, and therefore to compute the ionospheric combination we will need to convert the data to meters. To do this we first need to compute the wavelength of the data for each tracking channel

In [ ]:
# Get the wavelength for each tracking channel
wl_a = ch_a.get_wavelength(sat.constellation)
wl_b = ch_b.get_wavelength(sat.constellation)

Now computing the ionospheric combination is straightforward. We will create the LI combination as a **new column into the `DataFrame`** (to align the data with the epochs for later analysis).

In [ ]:
df_sat_li['li'] = df_sat_li['phase_a'] * wl_a - df_sat_li['phase_b'] * wl_b

We can now plot the LI combination using `matplotlib` plotting functions:

In [ ]:
# Close all previous figures
plt.close()

# Plot the LI against the time
plt.plot(df_sat_li['epoch'], df_sat_li['li'], '.')
plt.title(f'LI combination for {sat}')
plt.xlabel('Time [hour of 2014 doy 058]')
plt.ylabel('LI [m]')

## Compute $\Delta STEC$

We now will create another cell that contain the $\Delta STEC$, computed as the time derivative of the geometric free combination ($LI$). As shown before (see equation [](iono_comb_phase), besides the $STEC$, the geometry free combination of phases contain the phase ambiguities and the uncalibrated phase biases, which vanish with the time differentiation thanks to its constant nature. Therefore, the only term that will accompany the $\Delta STEC$ is the phase measurement noise ($\sqrt{2}\cdot \varepsilon_L$), which can be considered in the millimeter range.

To compute the time difference of the geometry free combination, we can simply use the [`numpy.diff` function](https://numpy.org/doc/stable/reference/generated/numpy.diff.html), which, again, will save us from using `for` loops thanks to its vectorized nature.

In [ ]:
d_li = np.diff(df_sat_li['li'])

We could now add this new time series into the dataframe. However, we must be careful due to a mismatch in the number of values: because of the difference operator, there is **one less value** in the `d_li` array, therefore, we need to add a sample manually.

In [ ]:
# Number of samples in the difference time series
len(d_li)

In [ ]:
# Number of samples in the original geometry combination time series
len(df_sat_li['li'])

We can insert (in this case prepend) a new sample in a numpy array using the insert method

In [ ]:
d_li = np.insert(d_li, 0, np.nan, axis=0)
len(d_li)

Now that we have homogenized sizes, we can create the column, but first we will perform some **data editing**, discarding (setting to NaN) all those LI values larger than a threshold (due to time gaps and cycle slips in the data). We can do this first by simple array indexing, to find those values larger than a threshold and then assign those rows to NaN.

In order to have an idea on which are the elements to remove, we can plot the histogram to have an idea on the distribution of samples and have an idea on where to place the thresholds


In [ ]:
plt.close()
_ = plt.hist(d_li)

From the histogram, it seems that samples below 2.5 could be safely discarded

In [ ]:
li_thresold = 1.0

d_li[d_li > +li_thresold] = np.nan
d_li[d_li < -li_thresold] = np.nan

The $\Delta STEC$ is simply the $\Delta LI$ scaled by the $\alpha$ factor, that depends on the frequencies used to compute the ionospheric combination

In [ ]:
f_a = ch_a.get_frequency(sat.constellation)
f_b = ch_b.get_frequency(sat.constellation)

alpha = 40.3 / (f_b * f_b) - 40.3 / (f_a * f_a)

df_sat_li['d_stec_tecu_per_s'] = d_li / alpha / 1.0e16

Now we can plot the STEC

In [ ]:
plt.close()
plt.plot(df_sat_li['epoch'], df_sat_li['d_stec_tecu_per_s'] * 60, marker='.')
plt.ylabel('Rate of STEC (TECU/min)')
plt.xlabel('Epoch')
plt.title('Rate of STEC')

## Compute ROTI


In order to compute the ROTI, we need to group the data in batches of a certain time period (e.g. 5 minutes) and compute the standard deviation for each of these batches. To do this we will use the [`resample` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html), but to use this we first need to set the epoch column as index of the `DataFrame`

In [ ]:
# Set the epoch as DataFrame time index. The resample method will use
# this index as basis for computation
df_sat_li.set_index('epoch', inplace=True)

In [ ]:
# Resample the dSTEC data every 5 minutes
df_sat_dstec_sampled = df_sat_li['d_stec_tecu_per_s'].resample('5T').std()
df_sat_dstec_sampled.head()

Now we can finally plot the data

In [ ]:

# Convert the pandas index (DateTimeIndex) to seconds for plotting
datetime_array = df_sat_dstec_sampled.index.to_numpy()

# Calculate the seconds of the day as a vectorized operation
seconds_of_day = (datetime_array - datetime_array.astype('datetime64[D]')) 

# Convert the timedelta values to seconds
seconds_of_day = seconds_of_day / np.timedelta64(1, 's')

plt.close()
plt.ylim(0,65)
plt.xlim(0,86400)
plt.xticks(np.linspace(0,86400, 7))
plt.yticks(np.arange(0,65, step=5))
plt.grid()
plt.plot(seconds_of_day, df_sat_dstec_sampled * 60)
plt.ylabel('Rate of TEC Index (TECU/min)')
plt.xlabel('Seconds of day 2014, doy 58')
plt.title('Rate of TEC Index for SEY1 station, GPS PRN26')

### ROTI with affordable receivers

The process can be repeated using an affordable receiver. In this case a
MEDEA computer (based on the u-blox ZED-F9P and a Talysmann antenna)

In [ ]:
rinex_file = '../assets/MTIC00ESP_R_20191221131_05H_01S_MO.rnx'

df = rinex.to_dataframe(rinex_file)

After loading the RINEX, let's perform some **basic data editing** to flag
phase breaks due to time gaps

In [ ]:
df = mark_time_gap(df)

Selection of the channels for which the ionospheric combination needs to be performed

In [ ]:
constellation = ConstellationId.GPS
channel_a = TrackingChannel.from_string('1C')
channel_b = TrackingChannel.from_string('2L')

Once the channels have been selected, the ionospheric (or geometry-free) combination
can be computed

In [ ]:
df_li_gps = compute_geometry_free(df, constellation, channel_a, channel_b)

# Preview the LI values
df_li_gps.head()

To compute the Slant Total Electron Content, we will need the $\alpha_{LI}$ coefficient (that transforms
LI to Slant Total Electron Content), which can be computed using the
frequency associated to the channel bands:

In [ ]:
f_a = channel_a.get_frequency(constellation)
f_b = channel_b.get_frequency(constellation)

alpha = 40.3 / (f_b * f_b) - 40.3 / (f_a * f_a)

In [ ]:
sat = Satellite.from_string('G29')
df_sat = df_li_gps[df_li_gps['sat'] == sat]

t = compute_decimal_hours(df_sat['epoch'])
plt.close()
plt.title(f"LI combination for {sat}")
plt.plot(t, df_sat['li_m'], '.')
plt.xlabel(f"Time [ hour of {df_sat.iloc[0]['epoch'].date()} ]")
plt.ylabel("LI [m]")

Once the LI combination is obtained, we are in the position of computing the $\Delta STEC$ as follows:

In [ ]:
df_li_gps['d_stec_tecu_per_s'] = df_li_gps.groupby('sat')['li_m'].diff() / alpha / 1.0e16

In [ ]:
df_sat = df_li_gps[df_li_gps['sat'] == sat]

t = compute_decimal_hours(df_sat['epoch'])
plt.close()
plt.title(f"Time difference of STEC for {sat}")
plt.plot(t, df_sat['d_stec_tecu_per_s'], '.')
plt.xlabel(f"Time [ hour of {df_sat.iloc[0]['epoch'].date()} ]")
plt.ylabel("DSTEC [TECU/s]")
plt.ylim(-1, 1)

Once the time difference of the STEC has been computed we can now proceed to
compute the standard deviation $\sigma$ for intervals of 5 minutes on a per-satellite
basis (i.e. definition of ROTI)

In [ ]:
# Set 'epoch' as the DataFrame index
df_li_gps.set_index('epoch', inplace=True)

In [ ]:
# Group the samples in 5 minute intervals and compute the sigma (i.e. ROTI)
df_roti = df_li_gps.groupby('sat').resample('5T').std(numeric_only=True).reset_index()

Plot the ROTI for a satellite

In [ ]:
df_sat = df_roti[df_roti['sat'] == sat]

plt.close()

t = compute_decimal_hours(df_sat['epoch'])

plt.close()
plt.ylim(0,65)
plt.yticks(np.arange(0,65, step=5))
plt.grid()
plt.ylabel('Rate of TEC Index (TECU/min)')
plt.xlabel(f"Time [ hour of {df_sat.iloc[0]['epoch'].date()} ]")
plt.title(f'ROTI for affordable receiver, satellite {sat}')
plt.plot(t, df_sat['d_stec_tecu_per_s'] *60)


In this particular case, no scintillation event was detected (also likely due
to the fact that the take was performed in mid latitude, in a quiet ionospheric
period). Note the peaks observed in the data. These are artifacts due to 
cycle slips.

Higher ROTI values have been also detected in other cases at low elevations. 
Care must be exercised in affordable receivers (i.e. atennas) since low
elevations might include multipath. Therefore, a **conservative elevation mask**
(e.g. $>15^\circ$) is recommended when processing affordable receivers